In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import sklearn.model_selection
from numpy.random import random
from sklearn.preprocessing import StandardScaler, MinMaxScaler

c0=sns.color_palette()[0]
c1=sns.color_palette()[1]
c2=sns.color_palette()[2]

cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

def points_plot(ax, Xtr, Xte, ytr, yte, clf, mesh=True, colorscale=cmap_light, 
                cdiscrete=cmap_bold, alpha=0.1, psize=10, zfunc=False, predicted=False):
    h = .02
    X=np.concatenate((Xtr, Xte))
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))

    #plt.figure(figsize=(10,6))
    if zfunc:
        p0 = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 0]
        p1 = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
        Z=zfunc(p0, p1)
    else:
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    ZZ = Z.reshape(xx.shape)
    if mesh:
        plt.pcolormesh(xx, yy, ZZ, cmap=cmap_light, alpha=alpha, axes=ax)
    if predicted:
        showtr = clf.predict(Xtr)
        showte = clf.predict(Xte)
    else:
        showtr = ytr
        showte = yte
    ax.scatter(Xtr[:, 0], Xtr[:, 1], c=showtr-1, cmap=cmap_bold, 
               s=psize, alpha=alpha,edgecolor="k")
    # and testing points
    ax.scatter(Xte[:, 0], Xte[:, 1], c=showte-1, cmap=cmap_bold, 
               alpha=alpha, marker="s", s=psize+10)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    return ax,xx,yy

def points_plot_prob(ax, Xtr, Xte, ytr, yte, clf, colorscale=cmap_light, 
                     cdiscrete=cmap_bold, ccolor=cm, psize=10, alpha=0.1):
    ax,xx,yy = points_plot(ax, Xtr, Xte, ytr, yte, clf, mesh=False, 
                           colorscale=colorscale, cdiscrete=cdiscrete, 
                           psize=psize, alpha=alpha, predicted=True) 
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=ccolor, alpha=.2, axes=ax)
    cs2 = plt.contour(xx, yy, Z, cmap=ccolor, alpha=.6, axes=ax)
    #plt.clabel(cs2, fmt = '%2.1f', colors = 'k', fontsize=14, axes=ax)
    return ax 

In [2]:
df = pd.read_csv('/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/data/processed/EDA.csv')


In [3]:
df.head().T

,0,1,2,3,4
season,2020,2020,2020,2020,2020
team,100 Thieves,Cloud9,Counter Logic Gaming,Dignitas,Evil Geniuses
kills_pg,10.898,15.6491,9.72973,9.28571,11.1667
deaths_pg,12.7551,9.26316,14.3784,11.0476,12.95
firstblood_pg,0.44898,0.561404,0.405405,0.47619,0.483333
assists_pg,24.7551,36.3684,24.4054,22.1667,26.1833
wardskilled_pg,49.0204,41.7719,46.5946,49.0952,47.0167
controlwardsbought_pg,42.9796,37.7895,44.1892,43.1667,47.3333
firsttower_pg,0.510204,0.754386,0.324324,0.404762,0.433333
towers_pg,5.61224,8,4.05405,4.61905,6.08333


In [4]:
df = df.set_index(['season','team'])

In [5]:
df.head(40)

kills_pg  deaths_pg  firstblood_pg  assists_pg  wardskilled_pg  controlwardsbought_pg  firsttower_pg  towers_pg  firsttothreetowers_pg  totalgold_pg  earnedgold_pg  goldspent_pg  minionkills_pg  monsterkills_pg  inhibitors_pg  Championship  Win_Percentage  Wins  Losses   kills  assists  firstblood  firstdragon  mountains  oceans  gamesplayed  deaths  wardskilled  controlwardsbought  firsttower  towers  firsttothreetowers  totalgold  earnedgold  goldspent  \
season team                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
2020   100 Thieves           10.897959  12.755102       0.448980   24.755102       49.020408              42.979592       0.510204   5.612245               0.367347  57166.857143   35227.510204  53744.040816      854.571429       205.959184       1.020408             0        0.408163  20.0    29.0   534.0   1213.0        22.0         24.0       30.0    39.0         49.0   625.0       2402.0              2106.0        25.0   275.0                18.0  2801176.0   1726148.0  2633458.0   
       Cloud9                15.649123   9.263158       0.561404   36.368421       41.771930              37.789474       0.754386   8.000000               0.754386  58425.631579   37973.438596  52644.842105      796.052632       221.631579       1.508772             0        0.771930  44.0    13.0   892.0   2073.0        32.0         38.0       42.0    38.0         57.0   528.0       2381.0              2154.0        43.0   456.0                43.0  3330261.0   2164486.0  3000756.0   
       Counter Logic Gaming   9.729730  14.378378       0.405405   24.405405       46.594595              44.189189       0.324324   4.054054               0.270270  57389.864865   34882.405405  54264.459459      896.594595       199.594595       0.540541             0        0.216216   8.0    29.0   360.0    903.0        15.0         13.0       14.0    16.0         37.0   532.0       1724.0              1635.0        12.0   150.0                10.0  2123425.0   1290649.0  2007785.0   
       Dignitas               9.285714  11.047619       0.476190   22.166667       49.095238              43.166667       0.404762   4.619048               0.333333  57318.428571   34914.142857  54120.047619      886.809524       217.928571       0.571429             0        0.357143  15.0    27.0   390.0    931.0        20.0         19.0       14.0    27.0         42.0   464.0       2062.0              1813.0        17.0   194.0                14.0  2407374.0   1466394.0  2273042.0   
       Evil Geniuses         11.166667  12.950000       0.483333   26.183333       47.016667              47.333333       0.433333   6.083333               0.516667  57701.816667   35752.533333  53983.800000      854.000000       208.866667       1.050000             0        0.483333  29.0    31.0   670.0   1571.0        29.0         34.0       33.0    36.0         60.0   777.0       2821.0              2840.0        26.0   365.0                31.0  3462109.0   2145152.0  3239028.0   
       FlyQuest              12.733333  11.000000       0.600000   31.266667       56.226667              40.693333       0.426667   6.480000               0.573333  60563.253333   38035.826667  55516.000000      892.133333       219.706667       1.280000             0        0.573333  43.0    32.0   955.0   2345.0        45.0         31.0       46.0    45.0         75.0   825.0       4217.0              3052.0        32.0   486.0                43.0  4542244.0   2852687.0  4163700.0   
       Golden Guardians       9.529412  12.000000       0.470588   22.588235       53.156863  

In [6]:
df.shape

(47, 38)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 47 entries, (2020, '100 Thieves') to (2014, 'Team SoloMid')
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   kills_pg               47 non-null     float64
 1   deaths_pg              47 non-null     float64
 2   firstblood_pg          47 non-null     float64
 3   assists_pg             47 non-null     float64
 4   wardskilled_pg         47 non-null     float64
 5   controlwardsbought_pg  47 non-null     float64
 6   firsttower_pg          47 non-null     float64
 7   towers_pg              47 non-null     float64
 8   firsttothreetowers_pg  47 non-null     float64
 9   totalgold_pg           47 non-null     float64
 10  earnedgold_pg          47 non-null     float64
 11  goldspent_pg           47 non-null     float64
 12  minionkills_pg         47 non-null     float64
 13  monsterkills_pg        47 non-null     float64
 14  inhibitors_pg       

In [8]:
key_features = ['kills_pg', 'towers_pg', 'assists_pg', 'firsttower_pg','Win_Percentage', 'earnedgold_pg', 'inhibitors_pg']

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the data into a training and test set.
Xlr, Xtestlr, ylr, ytestlr = train_test_split(df.drop(columns='Championship').values, 
                                              (df.Championship == 1).values,random_state=5)

clf = LogisticRegression()
# Fit the model on the training data.
clf.fit(Xlr, ylr)
# Print the accuracy from the testing data.
print(accuracy_score(clf.predict(Xtestlr), ytestlr))

1.0


/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [10]:
print(clf.predict(Xtestlr))

[False False False False False False False False False False False False]


In [11]:
print(clf.predict_proba(Xtestlr))

[[0.70064457 0.29935543]
 [0.84595602 0.15404398]
 [0.91476989 0.08523011]
 [0.79469513 0.20530487]
 [0.75465054 0.24534946]
 [0.95075713 0.04924287]
 [0.97143735 0.02856265]
 [0.96230765 0.03769235]
 [0.95724824 0.04275176]
 [0.90761009 0.09238991]
 [0.89798691 0.10201309]
 [0.95192731 0.04807269]]


In [12]:
scaler = StandardScaler()
scaler.fit(Xlr)
X_tr_scaled = scaler.transform(Xlr)
X_te_scaled = scaler.transform(Xtestlr)

In [13]:
clf.fit(X_tr_scaled, ylr)
# Print the accuracy from the testing data.
print(accuracy_score(clf.predict(X_te_scaled), ytestlr))

0.9166666666666666


In [14]:
print(clf.predict_proba(X_te_scaled))

[[5.49119117e-01 4.50880883e-01]
 [9.61837874e-01 3.81621258e-02]
 [9.48124980e-01 5.18750198e-02]
 [2.82582390e-01 7.17417610e-01]
 [5.50785230e-01 4.49214770e-01]
 [8.51627374e-01 1.48372626e-01]
 [9.99537493e-01 4.62507399e-04]
 [9.40275140e-01 5.97248603e-02]
 [9.89750215e-01 1.02497853e-02]
 [9.60903503e-01 3.90964973e-02]
 [9.74224610e-01 2.57753902e-02]
 [9.43584644e-01 5.64153558e-02]]


In [15]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

def cv_score(clf, x, y, score_func=accuracy_score):
    result = 0
    nfold = 5
    for train, test in KFold(nfold).split(x): # split data into train/test groups, 5 times
        clf.fit(x[train], y[train]) # fit
        result += score_func(clf.predict(x[test]), y[test]) # evaluate score function on held-out data
    return result / nfold # average

In [16]:
clf = LogisticRegression()
score = cv_score(clf, X_tr_scaled, ylr)
print(score)

0.5714285714285714


In [34]:
#the grid of parameters to search over
Cs = [0.001, 0.01, 0.1, 1, 10, 100]
max_score = 0

for C in Cs:
        clf = LogisticRegression(C=C)
        score = cv_score(clf, X_tr_scaled, ylr)
        if score > max_score:
            max_score = score
            best_C = C
        #print(C)
        print("{:.2f}".format(max_score))
        print(best_C)

0.80
0.001
0.80
0.001
0.80
0.001
0.80
0.001
0.80
0.001
0.80
0.001


/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [35]:
clf1 = LogisticRegression(C=.001)
clf1.fit(Xlr, ylr)
ypred = clf1.predict(Xtestlr)
print("{:.2f}".format(accuracy_score(clf1.predict(Xlr), ylr)))
print("{:.2f}".format(accuracy_score(ypred, ytestlr)))

0.89
0.92


/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [27]:
print(clf1.predict_proba(Xtestlr))

[[8.60711755e-01 1.39288245e-01]
 [6.87207358e-01 3.12792642e-01]
 [9.80816581e-01 1.91834188e-02]
 [4.89372809e-01 5.10627191e-01]
 [9.40770376e-01 5.92296244e-02]
 [9.83357268e-01 1.66427317e-02]
 [9.99145852e-01 8.54147614e-04]
 [9.71848928e-01 2.81510721e-02]
 [9.98447832e-01 1.55216770e-03]
 [9.53464681e-01 4.65353190e-02]
 [9.86918669e-01 1.30813312e-02]
 [9.37686798e-01 6.23132024e-02]]


In [19]:
from sklearn.model_selection import GridSearchCV

param_grid = {"C": [0.0001, 0.001, 0.1, 1, 10, 100]}
clf2 = LogisticRegression()

logreg_cv = GridSearchCV(clf2, param_grid, cv=5, scoring = 'accuracy')

# Fit it to the data
logreg_cv.fit(Xlr,ylr)

# Print the tuned parameters and score
print("Tuned Logistic Regression Parameters: {}".format(logreg_cv.best_params_)) 
print("Best score is {}".format(logreg_cv.best_score_))

logreg_cv.best_estimator_, logreg_cv.best_params_, logreg_cv.best_score_

/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

Tuned Logistic Regression Parameters: {'C': 0.0001}
Best score is 0.6285714285714286


/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/dankang/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/li

(LogisticRegression(C=0.0001, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 {'C': 0.0001},
 0.6285714285714286)

In [33]:
ypred = logreg_cv.predict(Xtestlr)
print("{:.2f}".format(accuracy_score(logreg_cv.predict(Xlr), ylr)))
print(accuracy_score(ypred, ytestlr))

0.80
1.0


In [28]:
print(logreg_cv.predict_proba(Xtestlr))

[[0.71789548 0.28210452]
 [0.87512268 0.12487732]
 [0.94832993 0.05167007]
 [0.84604431 0.15395569]
 [0.83550442 0.16449558]
 [0.96962628 0.03037372]
 [0.98478535 0.01521465]
 [0.96917952 0.03082048]
 [0.97703552 0.02296448]
 [0.94412203 0.05587797]
 [0.93477712 0.06522288]
 [0.96029905 0.03970095]]


In [21]:
from sklearn.model_selection import train_test_split

def do_classify(clf, parameters, indf, featurenames, targetname, target1val, standardize=False, train_size=0.8):
    subdf=indf[featurenames]
    if standardize:
        subdfstd=(subdf - subdf.mean())/subdf.std()
    else:
        subdfstd=subdf
    X=subdfstd.values
    y=(indf[targetname].values==target1val)*1
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=train_size)
    clf = cv_optimize(clf, parameters, Xtrain, ytrain)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print("Accuracy on training data: {:0.2f}".format(training_accuracy))
    print("Accuracy on test data:     {:0.2f}".format(test_accuracy))
    return clf, Xtrain, ytrain, Xtest, ytest

In [22]:
def cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=5):
    gs = sklearn.model_selection.GridSearchCV(clf, param_grid=parameters, cv=n_folds)
    gs.fit(Xtrain, ytrain)
    print("BEST PARAMS", gs.best_params_)
    best = gs.best_estimator_
    return best

In [23]:
clf_l, Xtrain_l, ytrain_l, Xtest_l, ytest_l  = do_classify(LogisticRegression(), 
                                                           {"C": [0.01, 0.1, 1, 10, 100]}, 
                                                           df, key_features, 'Championship',1)

BEST PARAMS {'C': 0.01}
Accuracy on training data: 0.81
Accuracy on test data:     1.00


In [24]:
clf_l.predict_proba(Xtest_l)

array([[0.73362068, 0.26637932],
       [0.88232842, 0.11767158],
       [0.89524643, 0.10475357],
       [0.73203732, 0.26796268],
       [0.82908692, 0.17091308],
       [0.78859758, 0.21140242],
       [0.88288983, 0.11711017],
       [0.79655089, 0.20344911],
       [0.88899882, 0.11100118],
       [0.7821503 , 0.2178497 ]])

In [25]:
#save our model
import os
import pickle

Pkl_Filename = "Lol_model.pkl"  
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(clf1, file)

In [26]:
#save our model
import os
import pickle
best_model = clf
    
modelpath = '/Users/dankang/Desktop/SpringBoard/Capstone2/Capstone2/models'
if not os.path.exists(modelpath):
    os.mkdir(modelpath)
Lol_path = os.path.join(modelpath, 'Lol_model.pkl')
if not os.path.exists(Lol_path):
    with open(Lol_path, 'wb') as f:
        pickle.dump(best_model, f)